# Machine Learning 101 in Python
### Led by Matteo Hessel, UCL ML Alumni, currently a Research Engineer at Google DeepMind, and Bojan Vujatovic, also UCL ML Alumni

1. Clone [this repository](https://github.com/bojanvujatovic/unsupervised-learning-workshop)
```
git clone https://github.com/bojanvujatovic/unsupervised-learning-workshop.git
cd unsupervised-learning-workshop    
```

2. Check if you have pip installed
```
pip --version  
```
If not, [download](https://bootstrap.pypa.io/get-pip.py) and run with python:
```
curl https://bootstrap.pypa.io/get-pip.py   (for Mac/Linux)
python get-pip.py 
``` 

3. Install all the dependecies necessary for the project (stored in requirements.txt) on your machine (if it fails  `pip install --upgrade pip`)
```
pip install -r requirements.txt
```

---

# Unsupervised Learning

Experience is provided as a stream of multidimensional data which may come in various forms: textual, visual, audio, etc…

**The goal is to find in the data some hidden structure, that can help us explain, compress or recode the data itself.**

### Focus on:
- Clustering (Explain, e.g. through a “generative” story)
- Feature Extraction (Recode, e.g. creating new attributes)

## Clustering
We group the data samples in such a way that elements in the same group are more similar to each other than to those in other groups.

The key element is: what does *similar* mean? How do we *measure* similarity?

- Clusters may focus on some external property (they look similar)
- Clusters may also identify some latent factor, such as all elements being generated by some common sub-process (a “generative story”)

### K-Means

#### Idea
Partition all data in k clusters, identified by a center. Clusters selected to minimize the sum of squared distances between each point and the center of the corresponding cluster.

$$ \operatorname{arg\,min}_s \sum\limits_{i=1}^k \sum\limits_{x \in S_i} \|x - \mu_i\|^2 $$

In order to minimize the sum of squared distances each data point is assigned to the cluster whose center is nearest.
How do we find the ideal locations of the centers?

### Algorithm

Randomly pick $K$ initial cluster centers.

**do**:

       Assign each element to the cluster that has the closest center.
       Recompute the centers as baricenter of the elements of the cluster: 
$$ m_i^{new} = \frac{1}{N_i} \sum\limits_{n \in N_i} x^n $$

**while** (centers are different from previous iteration)
